# Boosting

Después de haber realizado la comparación de 2 modelos (Knn Classifier  y SVM Classifier) no encontramos suficiente diferencia en el desempeño de estos dos modelos suficiente para seleccionar uno de ellos y desechar otro.

En esta sección intentaremos utilizar por lo menos uno de estos modelos ( SVM Classifier ) y aplicarle un proceso de boosting.

Asimismo probaremos otro modelo de boosting que no dependa de la dimensionalidad (Random Forest) para comparar los cuatro modelos.

Tomaremos el modelo de los cuatro:
+ Knn Classifier
+ SVM Classifier
+ Boost SVM 
+ Boost RandomForest 

In [16]:
from pandas import *
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import pandas.rpy.common as com

In [17]:
ro.r('datos.ori<-read.csv("../data/Miroculus.test.csv", header = FALSE, sep = ",",  dec=".", fill = TRUE)')
ro.r('datos<-datos.ori[-1]')
ro.r('corr<-cor(datos)')
pyCorr=com.load_data('corr')



Dividimos nuestros sets de datos:
* Train
* Valid
* Test  

De la misma forma que se hizo con los otros modelos

In [18]:
import random
random.seed(2255628424)
ro.r('val<-datos.ori[which(datos.ori["V1"]!="?"),]')
ro.r('m <- dim(val)[1]')
ro.r('testdtlength <- round(m/3)')
ro.r('sam <- sample(1:m, size = round(m/3), replace = FALSE, prob = rep(1/m, m)) ')
ro.r('datos.train <- val[-sam,]')
ro.r('datos.valid <- val[sam,]')
pyDatosTrain=com.load_data('datos.train')
pyDatosValid=com.load_data('datos.valid')

## Boost SVC
Realizaremos un proceso de boost (Adaboost) al modelo previamente definido esperando tener una mejor o por lo menos igual precisión.

In [19]:
import numpy as np


from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

XLearn = pyDatosTrain.ix[:, pyDatosTrain.columns != 'V1']
YLearn = pyDatosTrain['V1']


# creamos un modelo de Adaboost para nuestro SVC con los mismos parámetros obtenidos manualmente
mod = AdaBoostClassifier(SVC(gamma=.0001, C=1000),
                         algorithm="SAMME",
                         n_estimators=20)

mod.fit(np.asarray(XLearn),np.asarray(YLearn))

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          learning_rate=1.0, n_estimators=20, random_state=None)

In [20]:
from sklearn.metrics import accuracy_score
XValid = pyDatosValid.ix[:, pyDatosValid.columns != 'V1']
YValid = pyDatosValid['V1']

YPredic = mod.predict(np.asarray(XValid))

ac=accuracy_score(np.asarray(YValid),YPredic)
print("Accuracy:", "{:10.4f}".format(ac))


Accuracy:     0.8000


### Resultado
El proceso de boosting en el modelo de SVM __no__ muestra un aumento en la precisión del modelo (utilizando los mismos parámetros que tuvieron el mejor desempeño)

## Boost Random Forest
Intentaremos ahora entrenar un modelo de Random Forest, basado en teoría de la información realizando un boosting con diferentes parámetros tratando de encontrar un mejor desempeño del logrado hasta ahora.

In [21]:
import numpy as np


from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

XLearn = pyDatosTrain.ix[:, pyDatosTrain.columns != 'V1']
YLearn = pyDatosTrain['V1']


# Hacemos un un conjunto de entrenamientos con diferentes parámetros y comparamos los resultados
acMem=0.0
for i in [1,3,5,7,9] : ##max depth
    for j in [2,20,200,2000]: #estimators

        mod = AdaBoostClassifier(RandomForestClassifier(max_depth=i, n_estimators=j),
                                 algorithm="SAMME",
                                 n_estimators=100)

        mod.fit(np.asarray(XLearn),np.asarray(YLearn))

        YPredic = mod.predict(np.asarray(XValid))

        ac= accuracy_score(np.asarray(YValid),YPredic)

        if (ac > acMem) :
            acMem=ac
            depthMem=i
            estMem=j

print("Accuracy:", "{:10.4f}".format(ac))

Accuracy:     0.8667


###  Resultado
El proceso de boosting con RandomForest __tampoco__ nos muestra una mejor respuesta en cuanto a la precisión